### Funcs() & Libs for the project:

In [3]:
from itertools import chain, combinations_with_replacement as CwR, permutations as perm, combinations as combs
from html_table_parser import HTMLTableParser
from IPython.display import HTML 
from bs4 import BeautifulSoup
import urllib.request as ur
import pandas as pd
import numpy as np
import sys
import re

import locale
locale.setlocale(locale.LC_ALL, '')
num_prez = lambda num: f'{num:n}'


within_bracs = lambda s: re.search(r"\(([A-Za-z0-9_]+)\)", s) #Get content within brackets (sub-str)

remove_tags = lambda text: re.compile(r'<[^>]+>').sub(' ', str(text)) #Remove HTML tags -> str

get_int = lambda x: int(''.join(re.findall(r'\d+', x))) #Extract integers -> to list

get_float = lambda x: float(''.join(re.findall("[+-]?\d+\.\d+", x))) #Get floats -> list

dict_union = lambda *args: dict(chain.from_iterable(d.items() for d in args)) #Merge dicts

def m_replace(text, dic):
    
    for i, j in dic.items(): text = text.replace(i, j)
    return text


def get_tags(main_link,tag,class_):
    if class_ == '': extraction = [tag,{"class": class_}]
    else: extraction = [tag]
        
    get = []
    req = ur.Request(main_link,headers = {'User-Agent': 'okay'})
    url = ur.urlopen(req).read()
    soup = BeautifulSoup(url, "html.parser")
    #NORMALLY USE "title" instead of "target"! Using "target" for webometrics...
    for line in soup.find_all(*extraction): 
        
        target = remove_tags(line)
        get.append([target])
    
    return get

def get_links(main_link):
    get = []
    req = ur.Request(main_link,headers = {'User-Agent': 'okay'})
    url = ur.urlopen(req).read()
    soup = BeautifulSoup(url, "html.parser")
    #NORMALLY USE "title" instead of "target"! Using "target" for webometrics...
    for line in soup.find_all('a'): 
        txt_in_a,the_href = remove_tags(line),str(line.get('href'))
        get.append([txt_in_a,the_href])
    
    return get

#____________Displays DataFrames side-by-side (HTML)
def sbs(dfs): return HTML('<table><tr style="background-color:white;">'+
                    ''.join(['<td>'+table._repr_html_()+'</td>' for table in dfs])+
                                                                    '</tr></table>')
#___________________Display DataFrame in 3-pieces (^sbs() dependant)
def df_trisplit(df):
    trisplit_len = int(len(df)/3)
    len_last_rows = len(df)-trisplit_len
    
    return sbs([df.head(trisplit_len),df[trisplit_len:len_last_rows],df.tail(trisplit_len)])

#_________________________Get Website tables (HTML parser)
def webtables(target):
    headers = {'User-Agent': 'okay'}
    try:
        req = ur.Request(url=target)
        f = ur.urlopen(req)
        xhtml = f.read().decode('utf-8')
        p = HTMLTableParser()
        p.feed(xhtml)
        
        return p.tables 
    
    except:
        req = ur.Request(url=target,headers=headers)
        f = ur.urlopen(req)
        xhtml = f.read().decode('utf-8')
        p = HTMLTableParser()
        p.feed(xhtml)
        
        return p.tables 
        
#____________________Pandas Header = 1st row
def df_r1_header(df):
    
    new_header = df.iloc[0] 
    df = df[1:] 
    df.columns = new_header
    return df
#_____________________________University-list (with Ranks & Domains) from Webometrics
def get_webometrics(webo_url):
    a_tags, pre_table = get_links(webo_url), webtables(webo_url)
    
    start_list = [a_tags.index(i) for i in a_tags if 'Excellence Rank*' in i[0]][0]
    end_list = [a_tags.index(i) for i in a_tags if 'About Us' in i[0]][0]

    cleaned_links = [[i[0].strip().replace('&amp;','&'),i[1]] 
                     for i in a_tags[start_list+1:end_list] if i[0].strip()]

    pre_headers,tojoin_ = ['ranking','World Rank','University'],['University','Domain']

    cleaned_table = df_r1_header(pd.DataFrame(pre_table[0]))[pre_headers]

    links_table = pd.DataFrame(cleaned_links, columns = tojoin_)

    return pd.merge(cleaned_table,links_table,on='University')

#________________________Get school's Grade-letters - Chart 
def uni_gradesChart(uni):
     
    uni_dict = cad_univs[uni]
    uni_location = uni_dict['location']
    grade_keys = uni_dict['gradeConversions']
            
    if 'letter' in grade_keys.keys(): grades_chart = grade_keys['letter']
        
    elif 'percentage' and 'gpa' in grade_keys.keys(): grades_chart = grade_keys['gpa']
        
    else: grades_chart = grade_keys['percentage']
    
    if type(grades_chart) == str: grades_chart = commonRefs_dict[grades_chart]
    
    adjust_grades = ['/'.join(i) for i in grades_chart.values() if len(i) > 1]
    grades_list = list(chain(*[i for i in grades_chart.values() if len(i) == 1]))+adjust_grades
    
    switched_ = {i:j for i,j in zip(grades_list,grades_chart)} 
    
    return  switched_

def gpatabs(url):
    k = webtables(url)
    full_grades_df = df_r1_header(pd.DataFrame(k[0]))
    grades_dict = {i[0]:str(i[-1]) for i in full_grades_df.iloc[:, list(range(1)) + [-1]].values
                  if (i[-1].isdigit() or i[-1].replace('.','').isdigit())}
    
    for g,c in grades_dict.items(): grades_dict[g] = float(c)
    
    return {'gchart':full_grades_df,'letters':grades_dict}

## Webo-Areas

In [2]:
uSyria =  'https://www.webometrics.info/en/aw/Syrian%20Arab%20Republic'
uSaudi = 'https://www.webometrics.info/en/Asia/Saudi%20Arabia%20'
uPalestine = 'http://www.webometrics.info/en/aw/Palestinian%20Territories'

arab_world = ['ALGERIA', 'BAHRAIN', 'COMOROS', 'DJIBOUTI', 'EGYPT', 'IRAQ', 'JORDAN', 'KUWAIT', 'LEBANON', 
              'LIBYA', 'MAURITANIA', 'MOROCCO', 'OMAN', 'PALESTINIAN TERRITORIES', 'QATAR', 'SAUDI ARABIA', 
              'SOMALIA', 'SUDAN',' SYRIAN ARAB REPUBLIC','UNITED ARAB EMIRATES', 'YEMEN']
len(arab_world)

21

### 'QS World University Rankings 2020'

In [2]:
qs_main = 'https://www.topuniversities.com'
country_canada = '/universities/country/canada'
tops_in_canada = '/university-rankings-articles/world-university-rankings/top-universities-canada-2020'

get_qs = get_links(qs_main+country_canada) #Getting University-templates

#Extracting raw University-Box template details (between known indexes)
start_ind = get_qs.index([' X ', '/universities'])
end_ind = get_qs.index(['      ', '/where-to-study/north-america/canada/guide'])

raw_uni_list = get_qs[start_ind+1:end_ind]

#Initial preprocessing procedure for extracted Uni-Box details:
filter_uni_qstemps = {'\n ':'', '  ':',', ' #':',#', ' Size':',Size',
                      ' Postgrad':',Postgrad',' Research':','}

preprocess_uni_list = [m_replace(i[0].strip(),filter_uni_qstemps) for i in raw_uni_list]

#Extract and categorize Uni-Box templates with their varied details:
incomplete_dots, with_ranks, just_names = [],[],[]

for uni_temp in preprocess_uni_list:        
    if '...' in uni_temp: incomplete_dots.append(uni_temp.split(' ...')[0])
    
    elif 'QS' in uni_temp: with_ranks.append(uni_temp)
    
    elif ('QS' and '...' not in uni_temp) and 'Size' in uni_temp: 
        odd_uni_temp = uni_temp.split(',Size')[0]
        
        if ' Undergrad' in odd_uni_temp: 
            odd_uni_temp = odd_uni_temp.replace(' Undergrad',',Undergrad')
        just_names.append(odd_uni_temp)
        
    else: just_names.append(uni_temp)

#Next procedures: Extract all Uni-names and ranks via looping per category to a dictionary. 
#(Categories = Templates-with-ranks, Templates-with-just-names etc..)
qs_unis = {}

for i in with_ranks:
    comma_split = i.split(',')
    uni_name, rank, QS_title_yr = comma_split[0:3]
    
    qs_unis[uni_name] = {'rank':rank, 'qs_title':QS_title_yr, 'qs_url':''}
        
#Extracting from category with no ranks/qs-date-titles
for i in [i.split(',') for i in just_names]:
    uni_name = ' -'.join(i)
    if len(i) == 2 and 'Undergrad' in uni_name: uni_name = i[0]
    qs_unis[uni_name] = {'rank':'','qs_title':'QS World University Rankings 2020', 'qs_url':''}
    
#Category with incomplete names, full-titles extracted from respective school's QS-url
for i in raw_uni_list:  
    for j in incomplete_dots:
        if j in i[0]:
            uni_page_url = qs_main+i[1]
            get_full_title = get_tags(uni_page_url,'h1','')
            cleaned_title = get_full_title[0][0].strip().replace('&amp;','&')
            qs_unis[cleaned_title] = {'rank':'','qs_title':'QS World University Rankings 2020','qs_url':uni_page_url}
            
    for k in qs_unis: #Filling in QS-URLs for the rest of the Universities
        if k in i[0]:
            uni_page_url = qs_main+i[1]
            if qs_unis[k]['qs_url'] == '': qs_unis[k]['qs_url'] = uni_page_url

#WORLD RANKINGS - COUNTRY                
world_cad_qs = pd.DataFrame([[u,qs_unis[u]['rank']] for u in qs_unis],columns=['University','QS-rank'])
#world_qs.to_csv('world_qs_{}.csv'.format('country'))

df_trisplit(world_cad_qs)

### QS Rankings within Canada 2020

In [4]:
qs_cad_local = qs_main+tops_in_canada

cad_tops = webtables(qs_cad_local)
cad_rest_tops = get_tags(qs_cad_local,'li','')
irregular_ranks = get_tags(qs_cad_local,'p','')

filter_local_ranks = {'\xa0':'','.':''}

cad_rest_tops_list = [i.strip().split('–')[0].strip() for i in chain(*cad_rest_tops) if i.strip() != '']

cad_rest_start_ind = cad_rest_tops_list.index('Find us')
cad_rest_end_ind = cad_rest_tops_list.index('Sign in')

cad_rest_tops_list = cad_rest_tops_list[cad_rest_start_ind+1:cad_rest_end_ind]

cad_top_10 = [[i[0],i[-1]] for i in chain(*cad_tops) if len(i) > 1]


cad_rest_tops_list = [[str(x+1+int(cad_top_10[-1][0])),y] for x,y in enumerate(cad_rest_tops_list)]

preprocess_rest_ranks = [  [
    
    m_replace(i.strip().split('  ')[0],filter_local_ranks),    i.strip().split('  ')[1].strip()
            
                            ]  for i in chain(*irregular_ranks) if '=' in i.strip()]


odd_ranks = sorted([int(i[0].replace('=','')) for i in preprocess_rest_ranks])

s_oddranks, e_oddranks = min(odd_ranks), max(odd_ranks)

odd_ranks_range = range(s_oddranks, e_oddranks+len(odd_ranks)+1)

before_odd_ranks = [i for i in cad_rest_tops_list if int(i[0]) == s_oddranks][0]
before_odd_ranks_ind = cad_rest_tops_list.index(before_odd_ranks)

fix_ranks_order = cad_rest_tops_list[:before_odd_ranks_ind]+preprocess_rest_ranks
fix_last_ranks = [[str(int(i[0])+len(odd_ranks)), i[-1]] for i in cad_rest_tops_list[before_odd_ranks_ind:]]

local_ranks = cad_top_10+fix_ranks_order+fix_last_ranks
local_cad_qs = df_r1_header(pd.DataFrame(local_ranks)).reset_index(drop=True)

half_df_shape = int(local_cad_qs.shape[0]/2)
sbs([local_cad_qs.head(half_df_shape),local_cad_qs.tail(half_df_shape)])

,Canada Rank,University
0,1,University of Toronto
1,2,McGill University
2,3,University of British Columbia
3,4,University of Alberta
4,5,Université de Montréal
5,6,McMaster University
6,7,University of Waterloo
7,8,Western University
8,9,University of Calgary
9,10,Queen's University


### Joining World & Local QS Rankings - Canada

In [5]:
#Fixing English-French mismatch errors for Quebec University
get_quebec_local = local_cad_qs[local_cad_qs.University.str.contains('Québec')].index[0]
get_quebec_world = world_cad_qs[world_cad_qs.University.str.contains('Québec System')].index[0]
local_cad_qs.at[get_quebec_local,'University'] = world_cad_qs.at[get_quebec_world, 'University']

#University names that don't match between Local & World rank-DFs will be processed and joined:
local_in_world_qs = [i for i in world_cad_qs.University if i in local_cad_qs.University.values]
not_in_world_qs = [i for i in local_cad_qs.University if i not in local_in_world_qs]

df_checker = [(i,m_replace(i,{'University':'','Université':''}).strip()) for i in not_in_world_qs]

for uni in df_checker:
    get_match_index = world_cad_qs[world_cad_qs.University.str.contains(uni[-1])].index
    
    if list(get_match_index) == []: pass
    
    else: world_cad_qs.at[get_match_index[0],'University'] = uni[0]
        
print('Complete QS Rankings table for Canada!')
full_cad_qs = pd.merge(world_cad_qs,local_cad_qs, on='University', how='left').replace(np.nan,'')

display(full_cad_qs)

Complete QS Rankings table for Canada!


,University,QS-rank,Canada Rank
0,York University,#511-520,18
1,McGill University,#=35,2
2,Concordia University,#=462,17
3,Université de Montréal,#137,5
4,McMaster University,#=140,6
...,...,...,...
79,School of International Development and Global...,,
80,University of Ottawa - Telfer School of Manage...,,
81,Foreign Affairs and International Trade Canada,,
82,Osgoode Hall Law School International LLM Program,,


In [6]:
qs_copy_df = full_cad_qs.copy()

qs_cad_compiled_df = qs_copy_df[['University']]
qs_cad_compiled_df['QS-ranks'] = full_cad_qs['QS-rank'].astype(str)+' (cad-#'+full_cad_qs['Canada Rank'].astype(str)+')'

df_trisplit(qs_cad_compiled_df)

c:\users\faaris\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,University,QS-ranks
0,York University,#511-520 (cad-#18)
1,McGill University,#=35 (cad-#2)
2,Concordia University,#=462 (cad-#17)
3,Université de Montréal,#137 (cad-#5)
4,McMaster University,#=140 (cad-#6)
5,University of Waterloo,#=173 (cad-#7)
6,Western University,#211 (cad-#8)
7,University of Calgary,#233 (cad-#9)
8,Dalhousie University,#280 (cad-#11)
9,University of Toronto,#=29 (cad-#1)


## Keeping only record of the top 10 QS Unis

In [194]:
uni_french_eng = {'Montreal':'Montréal','University':'Université','of':'du','of':'de'}

with_complete_ranks = qs_cad_compiled_df[qs_cad_compiled_df['QS-ranks'].str.split().str.len() == 2].values
catch_top10 = [i for i in with_complete_ranks if get_int(i[-1].split()[-1]) in range(1,11)]

sorted_top10 = sorted(catch_top10, key = lambda x: get_int(x[-1].split()[-1]))

cad_qs_10 = pd.DataFrame(sorted_top10, columns= qs_cad_compiled_df.columns)

#cad_qs_10.to_csv('qs_cad_10.csv')
cad_qs_10

,University,QS-ranks
0,University of Toronto,#=29 (cad-#1)
1,McGill University,#=35 (cad-#2)
2,University of British Columbia,#51 (cad-#3)
3,University of Alberta,#113 (cad-#4)
4,Université de Montréal,#137 (cad-#5)
5,McMaster University,#=140 (cad-#6)
6,University of Waterloo,#=173 (cad-#7)
7,Western University,#211 (cad-#8)
8,University of Calgary,#233 (cad-#9)
9,Queen's University,#=239 (cad-#10)


### Getting top 10's from Times Higher Education (Canada)

In [195]:
the_cad = webtables('https://www.timeshighereducation.com/student/best-universities/best-universities-canada')
the_cad_df = df_r1_header(pd.DataFrame(chain(*the_cad)))

the_montreal = the_cad_df[the_cad_df.University.str.contains('Montreal')].index[0]
the_cad_df.at[the_montreal,'University'] = m_replace(the_cad_df.at[the_montreal,'University'],uni_french_eng)

the_cad_df.head(10)

,Canada Rank 2020,World University Rank 2020,University,Province,City
1,1,18,University of Toronto,Ontario,Toronto
2,2,34,University of British Columbia,British Columbia,Vancouver
3,3,42,McGill University,Quebec,Montreal
4,4,72,McMaster University,Ontario,Hamilton
5,5,85,Université de Montréal,Quebec,Montreal
6,6,=136,University of Alberta,Alberta,Edmonton
7,7,=141,University of Ottawa,Ontario,Ottawa
8,=8,201–250,University of Calgary,Alberta,Calgary
9,=8,201–250,University of Waterloo,Ontario,Waterloo
10,=8,201–250,Western University,Ontario,London


In [196]:
copy_the_df = the_cad_df.copy()

the_cad_10 = copy_the_df[['University','Province','City']]
the_cad_10['THE-ranks'] = copy_the_df['World University Rank 2020'].astype(str)+' (cad-#'+copy_the_df['Canada Rank 2020'].astype(str)+')'

the_cad_10 = the_cad_10[['University','THE-ranks','Province','City']].reset_index(drop=True)[:10]
the_cad_10

c:\users\faaris\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,University,THE-ranks,Province,City
0,University of Toronto,18 (cad-#1),Ontario,Toronto
1,University of British Columbia,34 (cad-#2),British Columbia,Vancouver
2,McGill University,42 (cad-#3),Quebec,Montreal
3,McMaster University,72 (cad-#4),Ontario,Hamilton
4,Université de Montréal,85 (cad-#5),Quebec,Montreal
5,University of Alberta,=136 (cad-#6),Alberta,Edmonton
6,University of Ottawa,=141 (cad-#7),Ontario,Ottawa
7,University of Calgary,201–250 (cad-#=8),Alberta,Calgary
8,University of Waterloo,201–250 (cad-#=8),Ontario,Waterloo
9,Western University,201–250 (cad-#=8),Ontario,London


### Same procedure with Webometrics

In [198]:
webo_cad = webtables('https://www.webometrics.info/en/North_america/Canada')
webo_cad_df = df_r1_header(pd.DataFrame(chain(*webo_cad)))
webo_cad_df.head(10)

,ranking,World Rank,University,Det.,Presence Rank*,Impact Rank*,Openness Rank*,Excellence Rank*
1,1,18,University of Toronto,,43,38,24,11
2,2,29,University of British Columbia,,47,46,28,31
3,3,55,McGill University,,121,66,57,59
4,4,69,University of Alberta,,271,86,68,75
5,5,90,University of Waterloo,,299,88,115,167
6,6,102,University of Calgary,,306,121,101,143
7,7,137,Simon Fraser University,,217,78,187,311
8,8,142,McMaster University,,652,174,37,164
9,9,165,Université de Montréal,,298,183,130,205
10,10,169,Western University (University of Western Onta...,,429,188,148,206


In [199]:
webo_copy = webo_cad_df.copy()

webo_cad_10 = webo_copy[['University']]
webo_cad_10['Webometrics'] = webo_copy['World Rank'].astype(str)+' (cad-#'+webo_copy['ranking'].astype(str)+')'

webo_cad_10 = webo_cad_10[['University','Webometrics']].reset_index(drop=True)[:10]
webo_cad_10

c:\users\faaris\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,University,Webometrics
0,University of Toronto,18 (cad-#1)
1,University of British Columbia,29 (cad-#2)
2,McGill University,55 (cad-#3)
3,University of Alberta,69 (cad-#4)
4,University of Waterloo,90 (cad-#5)
5,University of Calgary,102 (cad-#6)
6,Simon Fraser University,137 (cad-#7)
7,McMaster University,142 (cad-#8)
8,Université de Montréal,165 (cad-#9)
9,Western University (University of Western Onta...,169 (cad-#10)


## Merge the 3 Ranking-DFs

In [200]:
web3ranks_cad = pd.merge(pd.merge(cad_qs_10,webo_cad_10,on='University',how='outer'),the_cad_10,on='University',how='outer')

#web3ranks_cad.to_csv('3ranks_CANADA.csv')
web3ranks_cad

,University,QS-ranks,Webometrics,THE-ranks,Province,City
0,University of Toronto,#=29 (cad-#1),18 (cad-#1),18 (cad-#1),Ontario,Toronto
1,McGill University,#=35 (cad-#2),55 (cad-#3),42 (cad-#3),Quebec,Montreal
2,University of British Columbia,#51 (cad-#3),29 (cad-#2),34 (cad-#2),British Columbia,Vancouver
3,University of Alberta,#113 (cad-#4),69 (cad-#4),=136 (cad-#6),Alberta,Edmonton
4,Université de Montréal,#137 (cad-#5),165 (cad-#9),85 (cad-#5),Quebec,Montreal
5,McMaster University,#=140 (cad-#6),142 (cad-#8),72 (cad-#4),Ontario,Hamilton
6,University of Waterloo,#=173 (cad-#7),90 (cad-#5),201–250 (cad-#=8),Ontario,Waterloo
7,Western University,#211 (cad-#8),NaN,201–250 (cad-#=8),Ontario,London
8,University of Calgary,#233 (cad-#9),102 (cad-#6),201–250 (cad-#=8),Alberta,Calgary
9,Queen's University,#=239 (cad-#10),NaN,NaN,NaN,NaN


### Getting Tuition info. per School

In [8]:
cad_unis_tuition_url = 'https://www.univcan.ca/universities/facts-and-stats/tuition-fees-by-university/'
cad_tuition_webtable = webtables(cad_unis_tuition_url)

cad_tuition_df = df_r1_header(
                    pd.DataFrame([i for i in chain(*cad_tuition_webtable) if i])
                             ).reset_index(drop=True)
cad_tuition_df.to_csv('cad_$univcan.csv')
cad_tuition_df.head()

,University,Undergraduate - Canadian Students,Undergraduate - Foreign Students,Graduate - Canadian Students,Graduate - Foreign Students,Province
0,Memorial University of Newfoundland,"$2,550 - $3,330","$11,460 - $11,460","$1,905 - $3,717","$3,222 - $4,833",Newfoundland & Labrador
1,University of Prince Edward Island,"$6,270 - $6,270","$13,590 - $13,590","$4,215 - $7,650 *","$11,535 - $14,970 *",Prince Edward Island
2,Acadia University,"$7,650 - $8,933","$17,884 - $17,884","$7,600 - $8,883","$18,058 - $18,058",Nova Scotia
3,Université Sainte-Anne,"$6,504 - $7,787","$10,650 - $10,650","$6,504 - $7,787 *","$10,650 - $10,650 *",Nova Scotia
4,Dalhousie University,"$6,583 - $7,866","$17,661 - $19,134","$7,048 - $8,331","$18,126 - $19,599",Nova Scotia


In [9]:
#Matching any School that may be missed out within Tuition DF for our QS DF
cad_tuition_df.University = cad_tuition_df.University.str.replace('The University','University')

qs_match_tuition = {'Université de Moncton':'University of Moncton','Université Laval':'Laval University',
                       'Victoria University':'University of Victoria',
                      'Western University':'Western University',
                      'Laurentian University':'Laurentian University',
                      'University of Regina':'University of Regina',
                      'Athabasca University':'Athabasca University'}

qs_cad_tuition = cad_tuition_df.copy()

for k,v in qs_match_tuition.items(): 
    uni_ind = qs_cad_tuition[qs_cad_tuition.University.str.contains(k)].index[0]
    
    qs_cad_tuition.at[uni_ind,'University'] = v

In [10]:
#Collecting the Tuition-info for University of Quebec (System) by average tuition range of it's schools
quebec_unis = qs_cad_tuition[cad_tuition_df.University.str.contains('Québec')].copy()
quebec_unis.to_csv('quebec_unis_tuition.csv')

cad_tuition_floats = lambda x: [float(i) for i in m_replace(x,{'$':'',',':''}).split('-')]

quebec_sys,quebec_uni_names = [],[]
for i in range(1,5):
    float_list = quebec_unis.iloc[:,i].apply(cad_tuition_floats)
    avg_costs = sum([np.array(cost) for cost in float_list])/len(float_list)
    str_dollar_range = ' - '.join(['$'+num_prez(round(i)) for i in avg_costs])
    
    quebec_sys.append([float_list.name,str_dollar_range])

#Merged Tuition & QS DFs
merged_qs_tuition = pd.merge(qs_cad_compiled_df,qs_cad_tuition,on='University',how='left')

#Adding into the Marged DF the avg.-Tuition range values for University of Quebec (System)
quebec_sys_ind = merged_qs_tuition[merged_qs_tuition.University.str.contains('Québec System')].index[0]

for c,v in quebec_sys: merged_qs_tuition.at[quebec_sys_ind,c] = v

merged_qs_tuition.at[quebec_sys_ind,'Province'] = 'Québec'

#Drop other other Quebec Universities, as we have it's main (System) University (with the avg.-Tuition ranges)
delete_other_quebecs = merged_qs_tuition[merged_qs_tuition.University.str.contains('Québec') &  
                                        ~merged_qs_tuition.University.str.contains('Québec System')].copy()

merged_qs_tuition = merged_qs_tuition.drop(delete_other_quebecs.index)

#See results-------------------------
display(merged_qs_tuition)

merged_qs_tuition.loc[quebec_sys_ind]

,University,QS-ranks,Undergraduate - Canadian Students,Undergraduate - Foreign Students,Graduate - Canadian Students,Graduate - Foreign Students,Province
0,Concordia University,#=462 (cad-#17),"$2,544 - $7,940","$17,530 - $19,581","$1,908 - $5,955","$11,728 - $13,148",Québec
1,York University,#511-520 (cad-#18),"$6,118 - $6,118","$28,633 - $28,633","$2,871 - $2,871","$12,550 - $12,550",Ontario
2,McGill University,#=35 (cad-#2),"$2,544 - $7,940","$17,421 - $20,229","$2,544 - $7,940","$15,637 - $18,110",Québec
3,Université de Montréal,#137 (cad-#5),"$2,544 - $7,940","$17,421 - $19,459","$2,544 - $7,940","$15,637 - $17,421",Québec
4,McMaster University,#=140 (cad-#6),"$5,955 - $6,043","$26,981 - $30,744","$6,307 - $6,307","$16,761 - $17,096",Ontario


University                           University of Québec System
QS-ranks                                               (cad-#19)
Undergraduate - Canadian Students                $2,544 - $7,940
Undergraduate - Foreign Students               $17,422 - $18,781
Graduate - Canadian Students                     $2,544 - $7,940
Graduate - Foreign Students                    $15,934 - $17,421
Province                                                  Québec
Name: 62, dtype: object

### Eliminate 'School's from our DF
(departments as sub-section of a University or not 'actual' Universities/Colleges)

In [34]:
merged_qs_tuition[merged_qs_tuition['Undergraduate - Canadian\xa0Students'].isnull()]
merged_qs_tuition.to_csv('merged_qs_tuition.csv')

In [38]:
the_cad = webtables('https://www.timeshighereducation.com/student/best-universities/best-universities-canada')
the_cad_df = df_r1_header(pd.DataFrame(chain(*the_cad)))

In [39]:
the_cad_df

,Canada Rank 2020,World University Rank 2020,University,Province,City
1,1,18,University of Toronto,Ontario,Toronto
2,2,34,University of British Columbia,British Columbia,Vancouver
3,3,42,McGill University,Quebec,Montreal
4,4,72,McMaster University,Ontario,Hamilton
5,5,85,University of Montreal,Quebec,Montreal
6,6,=136,University of Alberta,Alberta,Edmonton
7,7,=141,University of Ottawa,Ontario,Ottawa
8,=8,201–250,University of Calgary,Alberta,Calgary
9,=8,201–250,University of Waterloo,Ontario,Waterloo
10,=8,201–250,Western University,Ontario,London
